# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!


### Text Classification
> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)

---
#### 

### 0. library

In [4]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud

In [33]:
## import sklearn
import numpy as np
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


### 1. 데이터 가져오기

In [5]:
import pickle

In [6]:
# 2. data preprocessing에서 저장한 결과를 불러오고 shape을 확인합니다.
with open('save_data.pickle', 'rb') as f:
    train_data, train_label = pickle.load(f)
    x_train, x_val = pickle.load(f)
    y_train, y_val = pickle.load(f)
    cv_x_train, cv_x_val = pickle.load(f)
    ngram_x_train, ngram_x_val = pickle.load(f)
    sequence_x_train, sequence_x_val, word_index = pickle.load(f)

In [7]:
x_train.shape

(14708,)

In [8]:
tf_idf = TfidfTransformer()

In [9]:
tf_idf_x_train = tf_idf.fit_transform(ngram_x_train)
tf_idf_x_val = tf_idf.transform(ngram_x_val)

In [10]:
import tensorflow as tf
from tensorflow import keras

In [11]:
tf_idf_x_train.shape

(14708, 10410)

In [50]:
from sklearn import metrics
#############
# Your Code #
#############
keras.backend.clear_session()

model = keras.models.Sequential()

model.add( keras.layers.Input( shape=(tf_idf_x_train.shape[1], )))

model.add( keras.layers.Dense(4096, activation='swish'))
model.add( keras.layers.Dense(4096, activation='swish'))

model.add( keras.layers.BatchNormalization())
model.add( keras.layers.Dropout(0.2))

model.add( keras.layers.Dense(2048, activation='swish'))
model.add( keras.layers.Dense(2048, activation='swish'))

model.add( keras.layers.BatchNormalization())
model.add( keras.layers.Dropout(0.2))

model.add( keras.layers.Dense(1024, activation='swish'))
model.add( keras.layers.Dense(1024, activation='swish'))

model.add( keras.layers.BatchNormalization())
model.add( keras.layers.Dropout(0.2))

model.add( keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              42643456  
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
batch_normalization (BatchNo (None, 4096)              16384     
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8

In [51]:
from tensorflow.keras.callbacks import EarlyStopping

In [52]:
es = EarlyStopping(monitor='val_loss', # 관측 대상 / 아직 지정 x
                   min_delta=0,        # 최소한 나빠지지 않으면 괜찮아
                   patience=5,         # 성능 개선되지 않는걸 얼마나 참을래? 5번
                   verbose=1,
                   restore_best_weights=True) 

In [53]:
model.fit(tf_idf_x_train.toarray(),
          y_train,
          validation_split=0.2,
          callbacks=[es],       # Early Stopping 적용
          verbose=1, epochs=50)

Epoch 1/50
368/368 [==============================] - 10s 25ms/step - loss: 0.7036 - accuracy: 0.7093 - val_loss: 0.4437 - val_accuracy: 0.7420
Epoch 2/50
368/368 [==============================] - 9s 24ms/step - loss: 0.2829 - accuracy: 0.8728 - val_loss: 0.3061 - val_accuracy: 0.8651
Epoch 3/50
368/368 [==============================] - 9s 24ms/step - loss: 0.2269 - accuracy: 0.9014 - val_loss: 0.3623 - val_accuracy: 0.8705
Epoch 4/50
368/368 [==============================] - 9s 24ms/step - loss: 0.2141 - accuracy: 0.8969 - val_loss: 0.3564 - val_accuracy: 0.8729
Epoch 5/50
368/368 [==============================] - 10s 28ms/step - loss: 0.1988 - accuracy: 0.9041 - val_loss: 0.3192 - val_accuracy: 0.8620
Epoch 6/50
368/368 [==============================] - 9s 24ms/step - loss: 0.2068 - accuracy: 0.8995 - val_loss: 0.3409 - val_accuracy: 0.8725
Epoch 7/50
368/368 [==============================] - 9s 24ms/step - loss: 0.2013 - accuracy: 0.9029 - val_loss: 0.3108 - val_accuracy: 0.86

In [24]:
NLP_pred = model.predict(tf_idf_x_val.toarray())

In [37]:
print(classification_report(np.round(NLP_pred, 0), y_val))

              precision    recall  f1-score   support

         0.0       0.83      0.81      0.82      1435
         1.0       0.88      0.89      0.89      2243

    accuracy                           0.86      3678
   macro avg       0.85      0.85      0.85      3678
weighted avg       0.86      0.86      0.86      3678



In [23]:
from sklearn.svm import SVC

In [24]:
svc = SVC()

In [ ]:
svc.fit(tf_idf_x_train.toarray(), y_train)

In [ ]:
svc.predict(tf_idf_x_val)

In [ ]:
accuracy_score(NB_pred, y_val)

In [ ]:
f1_score(NB_pred, y_val)

In [ ]:
print(classification_report(NB_pred, y_val))

In [55]:
from lightgbm import LGBMClassifier

In [64]:
lgbm = LGBMClassifier()

In [65]:
params = {"learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
          "max_depth" : [25, 50, 75],
          "num_leaves" : [100,300,500,900,1200],
          "n_estimators" : [100, 200, 300,500,800,1000],
          "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5]}

In [67]:
grid_lgbm = GridSearchCV(lgbm, params, scoring ='accuracy', cv=3, verbose=2)

In [ ]:
grid_lgbm.fit(tf_idf_x_train.toarray(), y_train)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=100; total time= 2.3min
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=100; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=100; total time= 3.3min
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=300; total time= 3.3min
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=300; total time= 2.9min
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=300; total time= 4.0min


In [59]:
lgbm_pred = lgbm.predict(tf_idf_x_val.toarray())

In [60]:
lgbm_pred 

array([1, 1, 1, ..., 0, 0, 0])

In [62]:
print(classification_report(lgbm_pred, y_val))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79      1561
           1       0.83      0.89      0.86      2117

    accuracy                           0.83      3678
   macro avg       0.83      0.82      0.83      3678
weighted avg       0.83      0.83      0.83      3678



### 2. Machine Learning(N-grams)

#### 2-1. Naive Bayes

In [12]:
# N-grams으로 vectorize한 데이터를 Naive Bayes 모델로 학습합니다.
NB = MultinomialNB()

NB.fit(tf_idf_x_train, y_train)

NB_pred = NB.predict(tf_idf_x_val)

In [13]:
NB_pred

array([1, 1, 1, ..., 0, 0, 0])

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [15]:
accuracy_score(NB_pred, y_val)

0.8045133224578576

In [16]:
f1_score(NB_pred, y_val)

0.851169530117988

In [17]:
print(confusion_matrix(NB_pred, y_val))

[[ 903  214]
 [ 505 2056]]


In [18]:
print(classification_report(NB_pred, y_val))

              precision    recall  f1-score   support

           0       0.64      0.81      0.72      1117
           1       0.91      0.80      0.85      2561

    accuracy                           0.80      3678
   macro avg       0.77      0.81      0.78      3678
weighted avg       0.83      0.80      0.81      3678



#### 2-2. 다른 모델들을 활용하여 학습합니다.

---

### 3. Deep Learning(Sequence)

In [20]:
# sequence 형태로 vectorize한 데이터를 딥러닝으로 학습합니다.
# Tip : 처음에는 embedding이 필요합니다
sequence_x_train

array([[   0,    0,    0, ...,  103,    3,   39],
       [   0,    0,    0, ...,    6,    5,   49],
       [   0,    0,    0, ..., 2191, 2192, 4865],
       ...,
       [   0,    0,    0, ...,  199,  361,   29],
       [   0,    0,    0, ...,   33,   31,   60],
       [   0,    0,    0, ...,    0,    0,    2]], dtype=int32)

#### 3-1. Conv1D

In [14]:
# Conv1D로 학습하고 모델 성능을 검증합니다.

#### 3-2. LSTM

In [ ]:
# LSTM으로 학습하고 모델 성능을 검증합니다.

In [48]:
import pickle as pc
import os
import numpy as np
# csv 모듈은 CSV 형식(쉼표로 구분된 표 형식) 데이터를 읽고 쓰는 클래스를 제공
import csv
import torch

In [7]:
import pickle as pc
import os
import numpy as np
# csv 모듈은 CSV 형식(쉼표로 구분된 표 형식) 데이터를 읽고 쓰는 클래스를 제공
import csv
import torch

# torch 버전 확인
print("Pytorch Version: ", torch.__version__)

# GPU 사용 가능한지 여부 확인
if torch.cuda.is_available():
    
    # PyTorch 에게 GPU 사용할거라고 알려주기
    device = torch.device("cuda")
    
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

Pytorch Version:  1.10.2+cu102
No GPU available, using the CPU instead.


In [8]:
# transformers 팩키지 설치
!pip install transformers

In [9]:
from transformers import BertTokenizer

# BERT tokenizer 불러오기
# do_lower_case : True이면 모두 소문자로 변환, False이면 대소문자 구분
print("Loading BERT tokenizer...")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [10]:
# 하나의 sentence에 대해 BertTokenizer 적용

# Print the original sentence.
print("Original: ", train_data[0])
print("Original: ", train_data[1])
print()

# Print the sentence split into tokens.
print("Tokenized: ", tokenizer.tokenize(train_data[0]))
print("Tokenized: ", tokenizer.tokenize(train_data[1]))
print()

# Print the sentence mapped to token ids.
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_data[0])))
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_data[1])))
print()

Original:  [Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA
Original:  [Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW]

Tokenized:  ['[', 'web', '##ᄇ', '##ᅡ', '##ᆯ', '##ᄉ', '##ᅵ', '##ᆫ', ']', '[UNK]', ':', '/', '/', 'lc', '##o', '.', 'jp', '/', 'ea']
Tokenized:  ['[', 'web', '##ᄇ', '##ᅡ', '##ᆯ', '##ᄉ', '##ᅵ', '##ᆫ', ']', '[UNK]', '?', '?', 'http', ':', '/', '/', 'bit', '##ly', '.', 'k', '##r', '/', 'br', '##gt', '##q', '[', 'f', '##w', ']']

Token IDs:  [1031, 4773, 29996, 30006, 30022, 29997, 30019, 30021, 1033, 100, 1024, 1013, 1013, 29215, 2080, 1012, 16545, 1013, 19413]
Token IDs:  [1031, 4773, 29996, 30006, 30022, 29997, 30019, 30021, 1033, 100, 1029, 1029, 8299, 1024, 1013, 1013, 2978, 2135, 1012, 1047, 2099, 1013, 7987, 13512, 4160, 1031, 1042, 2860, 1033]



In [11]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []

# For every sentence
for sent in train_data:
    # 'encode' will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the '[CLS]' token to the start.
    #   (3) Append the '[SEP]' token to the end.
    #   (4) Map tokens to their IDs.
    #   max_length : 문장의 최대길이
    #   encoded_sent : token IDs
    #---------------------------------------------------------------
    #      연습 (1)     tokenizer의 encode 함수를 호출해 주세요.  
    #---------------------------------------------------------------                
  
    encoded_sent = tokenizer.encode(sent, 
                                    add_special_tokens=True, # [CLS], [PAD], [SEP] 사용여부
                                    max_length = 64)
    
    # Add the encoded sentence to the list
    input_ids.append(encoded_sent)

# Print train data[0]
print("Original: ", train_data[0])
print()
print("Token IDs: ", input_ids[0])

# Print special tokens and tokenized sentence
print("\n[CLS] token: {:}, ID: {:}".format(tokenizer.cls_token, tokenizer.cls_token_id))
print("\n[PAD] token: {:}, ID: {:}".format(tokenizer.pad_token, tokenizer.pad_token_id))
print("\n[SEP] token: {:}, ID: {:}".format(tokenizer.sep_token, tokenizer.sep_token_id))
print("\nTokenized: ", tokenizer.convert_ids_to_tokens(input_ids[0]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  [Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA

Token IDs:  [101, 1031, 4773, 29996, 30006, 30022, 29997, 30019, 30021, 1033, 100, 1024, 1013, 1013, 29215, 2080, 1012, 16545, 1013, 19413, 102]

[CLS] token: [CLS], ID: 101

[PAD] token: [PAD], ID: 0

[SEP] token: [SEP], ID: 102

Tokenized:  ['[CLS]', '[', 'web', '##ᄇ', '##ᅡ', '##ᆯ', '##ᄉ', '##ᅵ', '##ᆫ', ']', '[UNK]', ':', '/', '/', 'lc', '##o', '.', 'jp', '/', 'ea', '[SEP]']


In [12]:
print("Max length: ", max([len(each) for each in input_ids]))

Max length:  64


In [13]:
import tensorflow as tf
print("Tensorflow version: {}".format(tf.__version__))

MAXLEN = 64
# post-sequence truncation, post-sequence padding
# padding value 0, type of output sequences long
input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, 
                                                          maxlen=MAXLEN, dtype="long", value=0, truncating="post", padding="post")

print("\nPadding is done.")

Tensorflow version: 2.4.2

Padding is done.


In [14]:
# Create attention masks
attention_masks = [] # 각 문장에 대한 attention mask 리스트를 저장

# 토큰 시퀀스에서 패딩에 해당하는 부분은 0, 패딩이 아닌 부분은 1을 넣은 mask를 생성
# 패딩 부분은 모델 내에서 Attention을 수행하지 않아 학습속도를 향상
# For every sentence
for sent in input_ids:
    # Create the attention mask.
    #  - If a token ID is 0, then it's padding, set the mask to 0.
    #  - If a token ID is not 0 ( > 0), then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)
print("\nAttention masking is done.")


Attention masking is done.


In [15]:
# Use train_test_split to split our data into train and validation sets for training
from sklearn.model_selection import train_test_split

# train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)
#     arrays : 분할시킬 데이터
#     test_size : 테스트 데이터셋의 비율 (default = 0.25)
#     random_state : 데이터 셔플 시 seed value. 호출할 때마다 동일한 학습 데이터, 테스트 데이터 셋을 생성하기 위해 설정
#     shuffle : 셔플 여부 (default = True)
#     stratify : 지정한 Data의 비율을 유지한다. 예를 들어, Label Set인 Y가 25%의 0과 75%의 1로 이루어진 Binary Set일 때
#     stratify=Y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%, 75%로 유지
# 반환값 : (학습 데이터, 테스트 데이터, 학습데이터 label, 테스트데이터 label)
# test_size=0.1로 설정, Use 90% for training and 10% for validation
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(input_ids, train_label, random_state=2018, test_size=0.1)

# Do the same for the masks.
train_masks, valid_masks, _, _ = train_test_split(attention_masks, train_label, random_state=2018, test_size=0.1)

# print train_inputs, valid_inputs
# 첫번째 학습데이터, 첫번째 attention mask를 출력해서 확인
print(train_inputs[:1])
print(train_masks[:1])

[[  101  1006  1455 30012 30025 29991 30011  1007   100   999   100  1012
   1047  2099  1013  1057 18153  2102  2094 29995 30014 29994 30013 29991
  30008 29996 30014   102     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [34]:
# Convert all inputs and labels into torch tensors, the required data type for our model.
# torch.tensor(data) -> tensor를 생성
train_inputs = torch.tensor(train_inputs)
valid_inputs = torch.tensor(valid_inputs)

train_masks = torch.tensor(train_masks)
valid_masks = torch.tensor(valid_masks)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct 

In [20]:
train_labels

17592    0
11950    1
10192    1
5310     0
1550     1
        ..
11228    1
7845     1
18890    1
15855    0
10200    0
Name: label, Length: 16547, dtype: int64

In [23]:
valid_labels

2135     1
15787    1
9577     1
8189     0
436      0
        ..
2609     0
13387    1
7351     1
11223    0
4757     1
Name: label, Length: 1839, dtype: int64

In [36]:
train_labels = torch.tensor(list(train_labels))
valid_labels = torch.tensor(list(valid_labels))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [37]:
# torch.utils.data : 파이토치의 데이터 로딩 유틸리티. 데이터셋, 데이터로더, 샘플러 등을 제공함
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.

batch_size = 32

#---------------------------------------------------------------
#      연습 (2) train_data, valid_data의 데이터 로더를 생성해 주세요.
#---------------------------------------------------------------   
# Create the DataLoader for our training set.
# torch.utils.data.TensorDataset(*tensors)
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)